<a href="https://colab.research.google.com/github/Feras-dev/si_harvest_ewaste_nn/blob/main/G2_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
File name: G2_notebook.ipynb
Authoers: Feras Alshehri
          Keerthana Gopal Seenivasan
          Adhaar Jaiswal
Description: 
last modified: 4/30/2021
"""

'\nFile name: G2_notebook.ipynb\nAuthoers: Feras Alshehri\n          Keerthana Gopal Seenivasan\n          Adhaar Jaiswal\nDescription: \nlast modified: 4/30/2021\n'

CSCI611 - project 

# Importing dependencies

In [2]:
import os
import zipfile
import keras

# Utility and Helper Functions

In [3]:
def get_file(url):
    """
    Fetches a zip file from the internet, and extract it locally.

    @input: url: str: string containing a full link to the zip file to be fetched.
    @ouput: True upon to successful execution. False otherwise.
    """
    
    file_name_i = url.rfind("/")
    file_name_f = len(url)
    file_name = url[file_name_i+1:file_name_f]

    print("Fetching {} from {}".format(file_name, url))

    try:
        filename = os.path.join(os.getcwd(), file_name)
        keras.utils.get_file(filename, url)
        print("Done fetching!")
    except:
        pass
        print("ERR: Failed fetching {} from {}".format(file_name, url))
        return False

    print("Extracting {}...".format(file_name))

    try:
        with zipfile.ZipFile(file_name, "r") as z_fp:
            z_fp.extractall("./")
        
        print("Done extracting!")
    except:
        pass
        print("ERR: Failed during extraction")
        return False

    return True


def output_to_csv_file(data, label, filename):
    """
    Write data to a csv file with a comma as a delimiter.
    @input: data: lkist of 
            label:
            filename: str: name of csv output file.
    @output:
    """
    for i in range(0, len(original_data)):
        pre_process = original_data[i].reshape((28*28,1))
        single_pic = []
        single_pic.append(label[0][i])
        for j in range(0,len(pre_process)):
            temp_pix = pre_process[j][0]
            single_pic.append(temp_pix)
        with open(filename,"a") as f:
            f_csv = csv.writer(f, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
            f_csv.writerow(single_pic)

#importing Data

In [4]:
if not (get_file("https://raw.github.com/Feras-dev/si_harvest_ewaste_nn/main/Dataset.zip")):
    exit()

Fetching Dataset.zip from https://raw.github.com/Feras-dev/si_harvest_ewaste_nn/main/Dataset.zip
Done fetching!
Extracting Dataset.zip...
Done extracting!


# Data pre-processing

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential

In [12]:
img_width, img_height = 150, 150

nb_train_samples = 3
nb_validation_samples = 20
epochs = 100
batch_size = 1

input_shape = (img_width,img_height,3)  

In [13]:
train_datagen = ImageDataGenerator(
    rescale = 1. /255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip=True
)

In [14]:
test_datagen = ImageDataGenerator(rescale=1. /255)

In [15]:
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

#Model initilization and creation

In [16]:
"""
set up model's initialization parameters.
"""
classes = ["si_posive", "si_negative"]
number_of_classes = len(classes)
